In [ ]:
import pandas as pd
from random import sample
import math

<h2> Make source code links for manual analysis </h2>

<h3>SQL</h3>

In [ ]:
projects=pd.read_csv("../Projects/SQL_projects_list.csv")
versions=pd.read_csv("../Data_Collection/SQL_project_versions.csv")
sdf=pd.read_csv("DAC_SQL_dataset_NoDuplicates_Classified.csv")

In [ ]:
projects.head(1)

In [ ]:
versions.head(2)

In [ ]:
sdf.info()

In [ ]:
#assign base url to each row
def get_url(project,projects):
    return projects[projects["project"]==project]["url"].head(1).tolist()[0]
def get_sha(project_version,versions):
    tks=project_version.split(";")
    return versions[(versions["project"]==tks[0]) & (versions["version"]==int(tks[1]))]["sha"].tolist()[0]
    #print("{}:{}\n".format(project,version))

In [ ]:
sdf["base_url"]=sdf["project"].apply(get_url, args=(projects,))

In [ ]:
sdf.head()

In [ ]:
sdf["version"]=sdf["version"].astype(str)
sdf["projVersion"]= sdf[["project","version"]].agg(';'.join, axis=1)
sdf.info()

In [ ]:
sdf["sha"]=sdf["projVersion"].apply(get_sha, args=(versions,))

In [ ]:
base=sdf.head(1)["base_url"].tolist()[0]
path=sdf.head(1)["path"].tolist()[0]
sha=sdf.head(1)["sha"].tolist()[0]
base +"/blob/"+sha+"/"+path
sdf["link"]=sdf["base_url"]+"/blob/"+sdf["sha"]+"/"+sdf["path"]

In [ ]:

sdf=sdf.drop(columns=["type","is_dac","base_url","projVersion"])

In [ ]:
sdf.info()

In [ ]:
sdf=sdf.drop(columns=["version","path","clean_comments"])

In [ ]:

sdf.head()
sdf["comment"]=sdf["comment"].apply(lambda X: X.strip())
sdf=sdf.sort_values(by=['topic'])

In [ ]:
sdf.reset_index(drop=True, inplace=True)

In [ ]:
sdf.to_csv("DAC_SQL_dataset_NoDuplicates_Classified_linked.csv", index=False)

<h3>NoSQL</h3>

In [ ]:
projects=pd.read_csv("../Projects/NoSQL_projects_list.csv")
versions=pd.read_csv("../Data_Collection/NoSQL_Project_versions.csv")
nsdf=pd.read_csv("DAC_NOSQL_dataset_NoDuplicates_Classified.csv")

In [ ]:
projects.head(1)

In [ ]:
versions.head(1)

In [ ]:
nsdf.info()

In [ ]:
nsdf.info()

In [ ]:
#assign base url to each row
def nget_url(project,projects):
    project="/".join(project.split("_"))
    return projects[projects["project"]==project]["url"].head(1).tolist()[0]
def nget_sha(project_version,versions):
    tks=project_version.split(";")
    return versions[(versions["project"]==tks[0]) & (versions["version"]==int(tks[1]))]["sha"].tolist()[0]
    #print("{}:{}\n".format(project,version))

In [ ]:
nsdf["base_url"]=nsdf["project"].apply(nget_url, args=(projects,))

In [ ]:
nsdf["version"]=nsdf["version"].astype(str)
nsdf["projVersion"]= nsdf[["project","version"]].agg(';'.join, axis=1)

In [ ]:
nsdf["sha"]=nsdf["projVersion"].apply(nget_sha, args=(versions,))

In [ ]:
nsdf["link"]=nsdf["base_url"]+"/blob/"+nsdf["sha"]+"/"+nsdf["path"]

In [ ]:
nsdf.head()
count_df=nsdf.groupby(['project']).count()
count_df["version"]

In [ ]:
nsdf=nsdf.drop(columns=["type","is_dac","base_url","projVersion"])

In [ ]:
nsdf.info()

In [ ]:
nsdf=nsdf.drop(columns=["version","path","clean_comments"])

In [ ]:
nsdf["comment"]=nsdf["comment"].apply(lambda X: X.strip())
nsdf=nsdf.sort_values(by=['topic'])

In [ ]:
nsdf.reset_index(drop=True, inplace=True)

In [ ]:
nsdf.head()

In [ ]:
nsdf.to_csv("DAC_NOSQL_dataset_NoDuplicates_Classified_linked.csv", index=False)

<h2>Generate samples</h2>

<h3>SQL</h3>

In [ ]:
#some exploration remove me

tdf= pd.read_csv("DAC_SQL_dataset_NoDuplicates_Classified_linked.csv")
tdf["group"]="sql"
ndf= pd.read_csv("DAC_NOSQL_dataset_NoDuplicates_Classified_linked.csv")
ndf["group"]="nosql"
df=  pd.concat([tdf,ndf],ignore_index=True)
df['id'] = df.index
df.head()

In [ ]:
tdf.info()

In [ ]:
ndf.info()

In [ ]:
#drop not needed columns
df=df.drop(columns=["sha"])

In [ ]:
df.tail()

In [ ]:
df["selected"]=False

In [ ]:
df.head()

In [ ]:
def remove_space (value):
    return value.strip()
df["comment"]=df["comment"].apply(remove_space)
df["id_comm"]=""+df["file_id"]+";"+df["comment"]
df["id_comm"]=df["id_comm"].apply(remove_space)
df.head(1)

In [ ]:

stopic_counts=tdf.groupby(["topic"]).size().reset_index(name='counts')
stopic_counts=stopic_counts.sort_values(by=['counts'], ascending=False)

def compute_strata_sample(strata_size,total_sample,total_population):
    return math.ceil ((strata_size*total_sample)/(total_population))
stopic_counts["strata_sample_size"]=stopic_counts["counts"].apply(compute_strata_sample,args=(173,4574))
stopic_counts["topic_size"]=stopic_counts["topic"].astype(str)+":"+stopic_counts["strata_sample_size"].astype(str)
stopic_counts

In [ ]:
#define a function that picks a sample
def pick_random_samples_per_topic(topic, df,sampleSize):
    indexes=df[df["topic"]==topic].index.tolist()
    return sample(indexes,sampleSize)

In [ ]:
topic_groups= stopic_counts["topic_size"].tolist()
sql_sampled_indexes=[]
for topic_size in topic_groups:
    tks=topic_size.split(":")
    sql_sampled_indexes.extend(pick_random_samples_per_topic(int(tks[0]),tdf,int(tks[1])))

In [ ]:

sample_tdf=tdf[tdf.index.isin(sql_sampled_indexes)]
sample_tdf.info()

In [ ]:
sample_tdf.info()

<h3>NOSQL</h3>

In [ ]:
#sample from the sql group
ndf= df[(df["group"]=="nosql") & (df["selected"]==False)]
ndf.info()

In [ ]:
ntopic_counts=ndf.groupby(["topic"]).size().reset_index(name='counts')
ntopic_counts=ntopic_counts.sort_values(by=['counts'], ascending=False)
#len(topic_counts.index)

ntopic_counts["strata_sample_size"]=ntopic_counts["counts"].apply(compute_strata_sample,args=(175,433))
ntopic_counts

In [ ]:
ntopic_counts["topic_size"]=ntopic_counts["topic"].astype(str)+":"+ntopic_counts["strata_sample_size"].astype(str)
ntopic_counts

In [ ]:
ntopic_groups= ntopic_counts["topic_size"].tolist()
nsql_sampled_indexes=[]
for topic_size in ntopic_groups:
    tks=topic_size.split(":")
    nsql_sampled_indexes.extend(pick_random_samples_per_topic(int(tks[0]),ndf,int(tks[1])))
len(nsql_sampled_indexes)

In [ ]:
nsample_tdf=ndf[ndf.index.isin(nsql_sampled_indexes)]

In [ ]:
#now combine the sampled comments
sample_tdf.info()

In [ ]:
sampld_combined= pd.concat([sample_tdf,nsample_tdf],ignore_index=True)
sampld_combined.info()
#sampld_combined=sampld_combined.drop(columns=["project","version","selected","id_comm","topic","file_id"])
sampld_combined.info()

In [ ]:
#save the sample data to csv
sampld_combined["number"]=sampld_combined.index
sampld_combined.to_csv("RQ3_Combined_Manual_Analysis_Dataset.csv", index=False)